In [1]:
#import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import wget
import pandas as pd
import locale

In [29]:
#connect to chrome driver
driver = webdriver.Chrome('E:/web scrapping/trends/chromedriver_win32/chromedriver.exe')

driver.get('https://www.instagram.com/accounts/login/')

C:\Users\Admin\AppData\Local\Temp/ipykernel_8576/4060695078.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:/web scrapping/trends/chromedriver_win32/chromedriver.exe')


In [30]:
#login to instagram
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
password.clear()


username.send_keys("basdas_returns")
password.send_keys("qwe123qwe")

Log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()

In [33]:
#search for your required trend
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

keyword = '#fashiontrends'
searchbox.send_keys(keyword)

my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
my_link.click()

In [ ]:
# finding images and their links
n_scrolls = 2
for i in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

anchors = driver.find_elements(By.TAG_NAME, "a")
anchors = [a.get_attribute('href') for a in anchors]
#narrow down all links to image links only
anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")]

print('Found ' + str(len(anchors)) + ' links to images')

anchors[:5]

images = []

#follow each image link and extract only image at index=1
for a in anchors:
    driver.get(a)
    time.sleep(5)
    img = driver.find_elements(By.TAG_NAME,'img')
    img = [i.get_attribute('src') for i in img]
    images.append(img[1])
    
images[:5]  

In [ ]:
#finding no of likes and comments per image
trending_score=[]
for i in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)


elements = WebDriverWait(driver, 25).until(EC.visibility_of_all_elements_located ((By.XPATH, "//a[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl _a6hd']")))
for element in elements:
    ActionChains(driver).move_to_element(element).perform()
    trending_score.append([my_elem.text for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, '//div[@class="_aacl _aacp _aacw _aad3 _aad6 _aade"]//span')))])

In [92]:
#reshaping and resizing the no of images and their respective trending scores
images = images[:21]
res1, res2 = map(list, zip(*trending_score[:21]))
a=[(x,y,z) for x,y,z in zip(images, res1, res2)]

In [97]:
df = pd.DataFrame(columns=['image-links','likes','Comment'])
for i in range(len(a)):
    df.loc[len(df)] = [a[i][0],a[i][1],a[i][2]]

In [98]:
df.shape

(21, 3)

In [156]:
df['likes'] = df['likes'].replace({'K': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['Comment'] = df['Comment'].astype(int)
df['trending score'] = df['likes']  + df['Comment']

In [159]:
df

,image-links,likes,Comment,trending score
0,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,562,36,598
1,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,9203,102,9305
2,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,49400,203,49603
3,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,246,2,248
4,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,597,10,607
5,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,3378,27,3405
6,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,3289,28,3317
7,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,1499,45,1544
8,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,819,24,843
9,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,31,3,34


In [160]:
df = df.sort_values(by=['trending score'], ascending=False)

In [165]:
df

,image-links,likes,Comment,trending score
15,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,49500,203,49703
2,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,49400,203,49603
13,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,9217,103,9320
1,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,9203,102,9305
5,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,3378,27,3405
18,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,3378,27,3405
16,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,3303,28,3331
6,https://instagram.fbom26-2.fna.fbcdn.net/v/t51...,3289,28,3317
19,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,1958,28,1986
20,https://instagram.fbom26-1.fna.fbcdn.net/v/t51...,1502,45,1547


In [ ]:
df.to_csv('trending_score.csv')